In [1]:
pip install jupyter numpy pandas matplotlib datasets tensorflow keras


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense


2025-06-17 11:30:52.092613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750140052.179676    5593 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750140052.199618    5593 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750140052.338672    5593 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750140052.338688    5593 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750140052.338689    5593 computation_placer.cc:177] computation placer alr

In [3]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train_texts = dataset['train']['text']
text = ' '.join(train_texts)


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Lowercase and filter out empty lines
text = text.lower().replace('\n', ' ')
text = ' '.join(text.split())  # Remove extra spaces

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1
print("Total unique words:", total_words)

# Create sequences
input_sequences = []
words = tokenizer.texts_to_sequences([text])[0]

for i in range(3, len(words)):
    n_gram_seq = words[i-3:i+1]
    input_sequences.append(n_gram_seq)


Total unique words: 66007


In [5]:
import numpy as np

input_sequences = np.array(input_sequences)
X = input_sequences[:, :-1]  # Input: first N-1 tokens
y = input_sequences[:, -1]   # Target: the next word (as integer label)

print("Input shape:", X.shape)
print("Output shape:", y.shape)
print("Sample input (token IDs):", X[0])
print("Sample label (token ID):", y[0])


Input shape: (1760504, 3)
Output shape: (1760504,)
Sample input (token IDs): [3799 3861  867]
Sample label (token ID): 18551


In [11]:
from tensorflow.keras.metrics import sparse_top_k_categorical_accuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

def top_5_accuracy(y_true, y_pred):
    return sparse_top_k_categorical_accuracy(y_true, y_pred, k=5)

model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))

# ✅ Compile with proper metrics
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy', top_5_accuracy]
)

model.build(input_shape=(None, X.shape[1]))
model.summary()



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 3, 100)         │     6,600,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 300)            │       301,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 66007)          │    19,868,107 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,770,007 (102.12 MB)

 Trainable params: 26,770,007 (102.12 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(X, y, epochs=5, batch_size=128, validation_split=0.1)


Epoch 1/5
  582/12379 ━━━━━━━━━━━━━━━━━━━━ 1:26:01 438ms/step - accuracy: 0.2517 - loss: 4.6432 - top_5_accuracy: 0.4299

KeyboardInterrupt: 

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

# Set up callbacks
checkpoint = ModelCheckpoint("checkpoint_model.h5", save_best_only=False)
csv_logger = CSVLogger("training_log.csv", append=True)

# Resume training from epoch 4 (since 3 completed)
history = model.fit(
    X, y,
    epochs=5,
    batch_size=128,
    validation_split=0.1,
    callbacks=[checkpoint, csv_logger],
    initial_epoch=4  # Change this if you're at a different epoch
)


Epoch 5/5
   83/12379 ━━━━━━━━━━━━━━━━━━━━ 1:27:21 426ms/step - accuracy: 0.2745 - loss: 4.5071 - top_5_accuracy: 0.4479

KeyboardInterrupt: 

In [30]:
model.save("final_lstm_model.h5")
print("Model saved as final_lstm_model.h5")
    

Model saved as final_lstm_model.h5


In [31]:
import pickle

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [32]:
model.save_weights("model.weights.h5")



In [34]:
import numpy as np

last_loss = 4.5071  # manually set your final loss value
perplexity = np.exp(last_loss)
print(f"Final Perplexity: {perplexity:.2f}")


Final Perplexity: 90.66
